In [ ]:
import os
project_dir = "/Volumes/mnt/Workspace/ai-trading-agents"
os.chdir(project_dir)

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import dotenv
dotenv.load_dotenv()

In [ ]:
from langchain.agents import create_agent
from src import middleware
from src import tools
from src.context import Context, get_context
import json
from src import db
from src.models import get_model

In [ ]:
model_name = "deepseek"
langchain_model = get_model(model_name)

In [ ]:
run_id = "b13e74d2-baef-40e0-a5f4-006f7019a13b"

In [ ]:
context = await get_context(run_id)